A simple example showing how to construct `Configuration` object from a pymatgen `Structure` object, using a coordination cutoff "recipe" to detect coordination polyhedra.

Problem:

TiOF<sub>2</sub> adopts the ReO<sub>3</sub> structure, with anion (O/F) disorder on the anion sub-lattice. For this example we want to find the Ti<i>X</i><sub>6</sub> octahedra in a 2&times;2&times;2 supercell.

In [1]:
from polyhedral_analysis.configuration import Configuration


Use the pymatgen `Poscar` class to load a VASP POSCAR and convert to a `Structure` object.

In [2]:
from pymatgen.io.vasp import Poscar
structure = Poscar.from_file('config_0000.poscar').structure
print(structure)

Full Formula (Ti8 O8 F16)
Reduced Formula: TiOF2
abc   :   7.596000   7.596000   7.596000
angles:  90.000000  90.000000  90.000000
Sites (32)
  #  SP        a     b     c
---  ----  -----  ----  ----
  0  Ti     0     0     0
  1  Ti     0     0     0.5
  2  Ti     0     0.5   0
  3  Ti     0     0.5   0.5
  4  Ti     0.5   0     0
  5  Ti     0.5   0     0.5
  6  Ti     0.5   0.5   0
  7  Ti     0.5   0.5   0.5
  8  O      0     0     0.25
  9  O      0     0     0.75
 10  O      0     0.5   0.25
 11  O      0     0.5   0.75
 12  O      0.5   0     0.25
 13  O      0.5   0     0.75
 14  O      0.5   0.5   0.25
 15  O      0.5   0.5   0.75
 16  F      0.25  0     0
 17  F      0.25  0     0.5
 18  F      0.25  0.5   0
 19  F      0.25  0.5   0.5
 20  F      0.75  0     0
 21  F      0.75  0     0.5
 22  F      0.75  0.5   0
 23  F      0.75  0.5   0.5
 24  F      0     0.25  0
 25  F      0     0.25  0.5
 26  F     -0     0.75  0
 27  F     -0     0.75  0.5
 28  F      0.5   0.25  0
 2

We first need to create a `PolyhedraRecipe` object that describes how to define polyhedra in our structure. 

In this example we use the `method="distance cutoff"` method, which assigns vertices to coordination polyhedra if these atoms are less than a cutoff distance $r$ from the central atom.

We also need to define the set of central atoms, and the set of coordination atoms, and the distance cutoff for coordination.

These four parameters are used to specify the following arguments for the `PolyhedraRecipe()` initialisation method:
- `method`
- `coordination_cutoff`
- `central_atom_list`
- `coordination_atom_list`

We first use the `Structure.indices_from_symbol()` method to get the index numbers of the central atoms, and of the vertex atoms.

TODO: Could write a class method that does this for you? e.g.
…

In [3]:
ti_indices = structure.indices_from_symbol('Ti')
of_indices = structure.indices_from_symbol('O') + structure.indices_from_symbol('F')

In [9]:
from polyhedral_analysis.polyhedra_recipe import PolyhedraRecipe
recipe = PolyhedraRecipe( method='distance cutoff', 
                          coordination_cutoff=3.0, 
                          central_atom_list=ti_indices,
                          coordination_atom_list=of_indices)
recipe

We can now use this recipe to create a particular polyhedra configuration from our `Structure`.

In [5]:
config = Configuration( structure=structure, recipes=[recipe] )

This `Configuration` contains the set of `CoordinationPolyhedra` that define each Ti<i>X</i><sub>6</sub> polyhedron.

In [10]:
config.polyhedra

[Coordination Polyhedron Ti
 0 [0. 0. 0.] Ti
 ----------
 8 [0.    0.    1.899] O
 9 [0.    0.    5.697] O
 16 [1.899 0.    0.   ] F
 20 [5.697 0.    0.   ] F
 24 [0.    1.899 0.   ] F
 26 [0.    5.697 0.   ] F, Coordination Polyhedron Ti
 1 [0.    0.    3.798] Ti
 ----------
 8 [0.    0.    1.899] O
 9 [0.    0.    5.697] O
 17 [1.899 0.    3.798] F
 21 [5.697 0.    3.798] F
 25 [0.    1.899 3.798] F
 27 [0.    5.697 3.798] F, Coordination Polyhedron Ti
 2 [0.    3.798 0.   ] Ti
 ----------
 10 [0.    3.798 1.899] O
 11 [0.    3.798 5.697] O
 18 [1.899 3.798 0.   ] F
 22 [5.697 3.798 0.   ] F
 24 [0.    1.899 0.   ] F
 26 [0.    5.697 0.   ] F, Coordination Polyhedron Ti
 3 [0.    3.798 3.798] Ti
 ----------
 10 [0.    3.798 1.899] O
 11 [0.    3.798 5.697] O
 19 [1.899 3.798 3.798] F
 23 [5.697 3.798 3.798] F
 25 [0.    1.899 3.798] F
 27 [0.    5.697 3.798] F, Coordination Polyhedron Ti
 4 [3.798 0.    0.   ] Ti
 ----------
 12 [3.798 0.    1.899] O
 13 [3.798 0.    5.697] O
 16 [1.